# Grid Search Prototype

In [4]:
import rospy
import time
import numpy as np
import sys
import yaml
import os
from os.path import exists
from ray import tune
import ray
from multiprocessing import Lock
from functools import partial
from config import ProfilerConfig
from pose_trajectory_evaluation import PoseTrajectoryEvaluation
from itertools import combinations, chain

%matplotlib inline
# %matplotlib notebook
%reload_ext autoreload
%autoreload 2

In [41]:
def read_grid_search_params(grid_search_params_file, tune_config):
    if not exists(grid_search_params_file):
        print('Found no grid search parameters.')
        return tune_config

    try:
        with open(grid_search_params_file) as f:
            data = yaml.load(f, Loader=yaml.FullLoader)
            for key, value in data.items():
                tune_config[key] = value
    except Exception as e:
        print('[Profiler] Unable to read and set grid search params at {file}'.format(file=grid_search_params_file))
        print(e)
    return tune_config


config_root = '/home/berlukas/Documents/workspace/darpa/src/darpa_subt_mapping/maplab/dependencies/internal/maplab_tools/maplab_profiles/config/'
profiling_grid_search_params_file = 'grid_search'
grid_search_params_file = config_root + profiling_grid_search_params_file + '.yaml'

print(f'Loading from {grid_search_params_file}')
config = {}
config = read_grid_search_params(grid_search_params_file, config)
print(f'Read the following: {config}')

Loading from /home/berlukas/Documents/workspace/darpa/src/darpa_subt_mapping/maplab/dependencies/internal/maplab_tools/maplab_profiles/config/grid_search.yaml
Read the following: {'vi_map_landmark_quality_min_observers': [2, 3, 4, 5], 'vi_map_landmark_quality_max_distance_from_closest_observer': [10, 20, 30, 40, 50], 'vi_map_landmark_quality_min_distance_from_closest_observer': [0.02, 0.05, 0.1, 0.3], 'vi_map_landmark_quality_min_observation_angle_deg': [2, 5, 10, 15]}


In [ ]:
def generate_permutations(config):
    np.array(np.meshgrid([1, 2, 3], [4, 5], [6, 7])).T.reshape(-1,3)
        
    
# perms = generate_permutations(config)


In [42]:
values = [np.arange(0,len(x)) for x in config.values()]
grid = np.meshgrid(*values)
combinations = np.array(grid).T.reshape(-1,len(values))

n_combinations = combinations.shape[0]
for i in range(0, n_combinations):
    print(f'combination {i+1} is {combinations[i,:]}')

combination 1 is [0 0 0 0]
combination 2 is [0 1 0 0]
combination 3 is [0 2 0 0]
combination 4 is [0 3 0 0]
combination 5 is [0 4 0 0]
combination 6 is [1 0 0 0]
combination 7 is [1 1 0 0]
combination 8 is [1 2 0 0]
combination 9 is [1 3 0 0]
combination 10 is [1 4 0 0]
combination 11 is [2 0 0 0]
combination 12 is [2 1 0 0]
combination 13 is [2 2 0 0]
combination 14 is [2 3 0 0]
combination 15 is [2 4 0 0]
combination 16 is [3 0 0 0]
combination 17 is [3 1 0 0]
combination 18 is [3 2 0 0]
combination 19 is [3 3 0 0]
combination 20 is [3 4 0 0]
combination 21 is [0 0 1 0]
combination 22 is [0 1 1 0]
combination 23 is [0 2 1 0]
combination 24 is [0 3 1 0]
combination 25 is [0 4 1 0]
combination 26 is [1 0 1 0]
combination 27 is [1 1 1 0]
combination 28 is [1 2 1 0]
combination 29 is [1 3 1 0]
combination 30 is [1 4 1 0]
combination 31 is [2 0 1 0]
combination 32 is [2 1 1 0]
combination 33 is [2 2 1 0]
combination 34 is [2 3 1 0]
combination 35 is [2 4 1 0]
combination 36 is [3 0 1 0]
c